In [1]:
import requests
import pandas as pd
import json
from datetime import datetime
import time
import os
from dotenv import load_dotenv


class TechStockSentimentExtractor:
    def __init__(self, api_key=None):
        self.api_key = api_key
        self.base_url = "https://www.alphavantage.co/query"
        
        # Top tech company tickers for 2025
        self.tech_tickers = {
            'AAPL': 'Apple Inc.',
            'MSFT': 'Microsoft Corporation', 
            'GOOGL': 'Alphabet Inc.',
            'AMZN': 'Amazon.com Inc.',
            'META': 'Meta Platforms Inc.',
            'TSLA': 'Tesla Inc.',
            'NVDA': 'NVIDIA Corporation',
            'NFLX': 'Netflix Inc.',
            'CRM': 'Salesforce Inc.',
            'AMD': 'Advanced Micro Devices'
        }
    
    def get_sentiment_data(self, ticker, limit=50, time_from=None, time_to=None):
        """Extract sentiment data using Alpha Vantage News Sentiment API"""
        
        params = {
            'function': 'NEWS_SENTIMENT',
            'tickers': ticker,
            'apikey': self.api_key,
            'limit': limit
        }
        
        if time_from:
            params['time_from'] = time_from
        if time_to:
            params['time_to'] = time_to
            
        try:
            response = requests.get(self.base_url, params=params)
            data = response.json()
            
            if 'Error Message' in data:
                print(f"Error for {ticker}: {data['Error Message']}")
                return None
            elif 'Note' in data:
                print(f"Rate limit reached: {data['Note']}")
                return None
                
            return data
            
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
            return None

    def process_sentiment_score(self, news_data, ticker):
        """Calculate aggregate sentiment score from news data"""
        if not news_data or 'feed' not in news_data:
            return None
            
        total_score = 0
        count = 0
        article_count = len(news_data['feed'])
        
        # Extract sentiment scores for the specific ticker
        for article in news_data['feed']:
            if 'ticker_sentiment' in article:
                for ticker_info in article['ticker_sentiment']:
                    if ticker_info.get('ticker') == ticker:
                        sentiment_score = float(ticker_info.get('ticker_sentiment_score', 0))
                        total_score += sentiment_score
                        count += 1
        
        if count == 0:
            return None
            
        avg_sentiment = total_score / count
        
        # Categorize sentiment based on Alpha Vantage scale
        if avg_sentiment >= 0.15:
            category = "Bullish"
        elif avg_sentiment <= -0.15:
            category = "Bearish"
        else:
            category = "Neutral"
            
        return {
            'ticker': ticker,
            'avg_sentiment_score': round(avg_sentiment, 4),
            'sentiment_category': category,
            'articles_analyzed': count,
            'total_articles': article_count,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

    def extract_all_tech_sentiment(self, save_to_csv=True):
        """Extract sentiment data for all top tech companies"""
        results = []
        
        print("Starting sentiment extraction for top tech companies...")
        
        for i, (ticker, company_name) in enumerate(self.tech_tickers.items(), 1):
            print(f"Processing {i}/{len(self.tech_tickers)}: {ticker} ({company_name})")
            
            datetime_now = datetime.now()

            sentiment_data = self.get_sentiment_data(ticker, limit=100, time_from=datetime_now)
            
            if sentiment_data:
                processed_data = self.process_sentiment_score(sentiment_data, ticker)
                if processed_data:
                    processed_data['company_name'] = company_name
                    results.append(processed_data)
                    print(f"  ✓ Sentiment: {processed_data['avg_sentiment_score']} ({processed_data['sentiment_category']})")
            
            # Respect API rate limits (12 seconds between calls for free tier)
            if i < len(self.tech_tickers):
               time.sleep(12)
        
        # Convert to DataFrame and save as CSV
        df = pd.DataFrame(results)
        
        if save_to_csv and not df.empty:
            filename = f'tech_stock_sentiment_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
            df.to_csv(filename, index=False)
            print(f"\nData saved to: {filename}")
        
        return df


In [2]:
load_dotenv()
extractor = TechStockSentimentExtractor(api_key=os.getenv("ALPHA_VANTAGE_API_KEY"))

In [3]:
sentiment_df = extractor.extract_all_tech_sentiment()

Starting sentiment extraction for top tech companies...
Processing 1/10: AAPL (Apple Inc.)
  ✓ Sentiment: 0.1594 (Bullish)
Processing 2/10: MSFT (Microsoft Corporation)
  ✓ Sentiment: 0.1261 (Neutral)
Processing 3/10: GOOGL (Alphabet Inc.)
Processing 4/10: AMZN (Amazon.com Inc.)
  ✓ Sentiment: 0.2452 (Bullish)
Processing 5/10: META (Meta Platforms Inc.)
  ✓ Sentiment: 0.0969 (Neutral)
Processing 6/10: TSLA (Tesla Inc.)
  ✓ Sentiment: 0.0661 (Neutral)
Processing 7/10: NVDA (NVIDIA Corporation)
  ✓ Sentiment: 0.1492 (Neutral)
Processing 8/10: NFLX (Netflix Inc.)
  ✓ Sentiment: 0.2239 (Bullish)
Processing 9/10: CRM (Salesforce Inc.)
  ✓ Sentiment: 0.0655 (Neutral)
Processing 10/10: AMD (Advanced Micro Devices)
  ✓ Sentiment: 0.2371 (Bullish)

Data saved to: tech_stock_sentiment_20250701_181522.csv


In [4]:
if not sentiment_df.empty:
    print("Sentiment Analysis Results:")
    print(sentiment_df[['ticker', 'company_name', 'avg_sentiment_score', 'sentiment_category']])

Sentiment Analysis Results:
  ticker            company_name  avg_sentiment_score sentiment_category
0   AAPL              Apple Inc.               0.1594            Bullish
1   MSFT   Microsoft Corporation               0.1261            Neutral
2   AMZN         Amazon.com Inc.               0.2452            Bullish
3   META     Meta Platforms Inc.               0.0969            Neutral
4   TSLA              Tesla Inc.               0.0661            Neutral
5   NVDA      NVIDIA Corporation               0.1492            Neutral
6   NFLX            Netflix Inc.               0.2239            Bullish
7    CRM         Salesforce Inc.               0.0655            Neutral
8    AMD  Advanced Micro Devices               0.2371            Bullish
